## Задание 

1. Добавить метод *fit_predict* в класс AdaBoost_custom и опцию base_function для пользовательского определения базовой функции.
2. Используя функцию для обучения алгоритма <a href=https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html> AdaBoost</a> решить задачу прогнозирования вероятности выполнения погрузки. Подобрать оптимальную базовую функцию (взять по одному методу из линейных, логических и метрических методов классификации) и ее оптимальные гиперпараметры с помощью поиска по сетке.
3. Оценить важность признаков. Вывести топ 10 наиболее важных признаков.

In [13]:
df = pd.read_excel('orders_short_v2.xlsx')

df = df[(df['Контрибуция']>-60000000) & (df['Среднее время между редакциями']>=0)]

In [14]:
df_rule = df['Дата отправления']>='2023-09-01'

In [15]:
categorial = ['Оператор', 'Учетное направление', 'Кластер назначения', 'Кластер отправления']

In [16]:
df.drop(['Unnamed: 0',
         'Идентификатор погрузки',
         'Дата редактирования', 
         'Дата отправления', 
         'Дата отправления факт', 
         'Количество редакций после отправки',
         'Длительность афтершока', 
         'Код назначения', 
         'Код отправления', 
         'Идентификатор переноса'], axis = 1, inplace = True)

In [17]:
numerical = ['версия погрузки', 
             'Количество вагонов', 
             'Контрибуция', 
             'Длительность жизни', 
             'Время до отправки',
             'Количество отмен', 
             'Количество переносов',
             'Среднее время между редакциями']

In [18]:
X_train = df[~df_rule].drop('Результат', axis = 1)
y_train = df['Результат'][~df_rule]

X_test = df[df_rule].drop('Результат', axis = 1)
y_test = df['Результат'][df_rule]

In [19]:
for col in categorial:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])
        
for col in numerical:
    scaler = StandardScaler()
    X_train[numerical] = scaler.fit_transform(X_train[numerical])
    X_test[numerical] = scaler.transform(X_test[numerical])

In [20]:
log_reg_grid = GridSearchCV(
    estimator=AdaBoostClassifier(random_state=42),
    
    param_grid={
        'estimator': [LogisticRegression(solver=solver, C=C) #penalty=p) 
                      for solver in ['lbfgs', 'liblinear', 'saga'] 
                      for C in [0.01, 0.1, 1, 3, 5, 10]],
                      #for p in ['l2', 'l1', 'elasticnet']],
        'n_estimators': [10, 50, 70, 100]
    },
    
    scoring='roc_auc',
    cv=5,
    n_jobs=-1,
    verbose=3
)

In [21]:
log_reg_grid.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


GridSearchCV(cv=5, estimator=AdaBoostClassifier(random_state=42), n_jobs=-1,
             param_grid={'estimator': [LogisticRegression(C=0.01),
                                       LogisticRegression(C=0.1),
                                       LogisticRegression(C=1),
                                       LogisticRegression(C=3),
                                       LogisticRegression(C=5),
                                       LogisticRegression(C=10),
                                       LogisticRegression(C=0.01,
                                                          solver='liblinear'),
                                       LogisticRegression(C=0.1,
                                                          solver='liblinear'),
                                       LogisticRe...
                                                          solver='liblinear'),
                                       LogisticRegression(C=10,
                                                          solver='liblinear'),
                                       LogisticRegression(C=0.01,
                                                          solver='saga'),
                                       LogisticRegression(C=0.1, solver='saga'),
                                       LogisticRegression(C=1, solver='saga'),
                                       LogisticRegression(C=3, solver='saga'),
                                       LogisticRegression(C=5, solver='saga'),
                                       LogisticRegression(C=10, solver='saga')],
                         'n_estimators': [10, 50, 70, 100]},
             scoring='roc_auc', verbose=3)

In [22]:
log_reg_grid.best_params_

{'estimator': LogisticRegression(C=10, solver='liblinear'), 'n_estimators': 50}

In [23]:
log_reg_grid.best_score_

0.7955278544849119

AdaBoost не имеет встроенной оценки важности признаков, поэтому для модели адаптивного бустинга с базовой моделью логистической регрессии будем извлекать коэффициенты для каждой модели логистической регрессии, находить их сумму и делить на общее количество базовых моделей

In [24]:
model_log_reg = AdaBoostClassifier(
    estimator=LogisticRegression(solver='liblinear', C=10),
    n_estimators=50,
    random_state=42
)

In [25]:
model_log_reg.fit(X_train, y_train)

AdaBoostClassifier(estimator=LogisticRegression(C=10, solver='liblinear'),
                   random_state=42)

In [26]:
log_regs = model_log_reg.estimators_

# Создаем массив для записи коэффициентов для признаков
feature_coefs = np.zeros(X_train.shape[1])

In [27]:
for log_reg in log_regs:
    feature_coefs += np.abs(log_reg.coef_[0])
feature_coefs /= len(log_regs)

In [28]:
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': feature_coefs
})

In [29]:
importance_df = importance_df.sort_values(by='importance', ascending=False)
importance_df.head(10)

,feature,importance
0,версия погрузки,0.107900
9,Количество отмен,0.094707
10,Количество переносов,0.052402
8,Время до отправки,0.035463
7,Длительность жизни,0.020381
5,Количество вагонов,0.015925
6,Контрибуция,0.013357
11,Среднее время между редакциями,0.011620
2,Учетное направление,0.009074
4,Кластер отправления,0.005324


In [30]:
fig = px.bar(importance_df, x='feature', y='importance')
fig.show()

В случае с решающими деревьями, будем использовать встроенную оценку важности для каждого решающего дерева, суммировать эти оценки и делить на количество деревьев, тем самым находить среднее значение важности для каждого признака по всем деревьям в модели

In [31]:
decision_tree_grid = GridSearchCV(
    estimator=AdaBoostClassifier(random_state=42),
    param_grid={
        'estimator': [DecisionTreeClassifier(max_depth=depth, min_samples_leaf=leaf) 
                      for depth in [3, 5, 7, 10]
                      for leaf in [1, 3, 5, 7]],
        'n_estimators': [10, 50, 70, 100]
    },
    scoring='roc_auc',
    cv=5,
    n_jobs=-1,
    verbose=3
)

In [32]:
decision_tree_grid.fit(X_train, y_train)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


GridSearchCV(cv=5, estimator=AdaBoostClassifier(random_state=42), n_jobs=-1,
             param_grid={'estimator': [DecisionTreeClassifier(max_depth=3),
                                       DecisionTreeClassifier(max_depth=3,
                                                              min_samples_leaf=3),
                                       DecisionTreeClassifier(max_depth=3,
                                                              min_samples_leaf=5),
                                       DecisionTreeClassifier(max_depth=3,
                                                              min_samples_leaf=7),
                                       DecisionTreeClassifier(max_depth=5),
                                       DecisionTreeCl...
                                       DecisionTreeClassifier(max_depth=7,
                                                              min_samples_leaf=5),
                                       DecisionTreeClassifier(max_depth=7,
                                                              min_samples_leaf=7),
                                       DecisionTreeClassifier(max_depth=10),
                                       DecisionTreeClassifier(max_depth=10,
                                                              min_samples_leaf=3),
                                       DecisionTreeClassifier(max_depth=10,
                                                              min_samples_leaf=5),
                                       DecisionTreeClassifier(max_depth=10,
                                                              min_samples_leaf=7)],
                         'n_estimators': [10, 50, 70, 100]},
             scoring='roc_auc', verbose=3)

In [33]:
decision_tree_grid.best_params_

{'estimator': DecisionTreeClassifier(max_depth=3, min_samples_leaf=7),
 'n_estimators': 100}

In [34]:
decision_tree_grid.best_score_

0.840508115878112

In [35]:
model_tree = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=3, min_samples_leaf=7),
    n_estimators=100,
    random_state=42
)

In [36]:
model_tree.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=3,
                                                    min_samples_leaf=7),
                   n_estimators=100, random_state=42)

In [37]:
trees = model_tree.estimators_
feature_importances = np.zeros(X_train.shape[1])

In [38]:
for tree in trees:
    feature_importances += tree.feature_importances_
feature_importances /= len(trees)

In [39]:
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': feature_importances
})

In [40]:
importance_df = importance_df.sort_values(by='importance', ascending=False)
importance_df.head(10)

,feature,importance
6,Контрибуция,0.161812
8,Время до отправки,0.134427
9,Количество отмен,0.121804
0,версия погрузки,0.108958
1,Оператор,0.096589
7,Длительность жизни,0.078365
3,Кластер назначения,0.072814
11,Среднее время между редакциями,0.069099
5,Количество вагонов,0.066624
10,Количество переносов,0.034263


In [41]:
fig = px.bar(importance_df, x='feature', y='importance')
fig.show()